In [1]:
import pandas as pd
import seaborn as sns

In [2]:
def makeAndCleanDF ():
    avengersEndgame = pd.read_csv('Avengers Endgame.csv')
    forrestGump = pd.read_csv('Forrest Gump.csv')
    johnWickChapterThree = pd.read_csv('John Wick Chapter 3.csv')
    joker = pd.read_csv('Joker.csv')
    morbius = pd.read_csv('Morbius.csv')
    pulpFiction = pd.read_csv('Pulp Fiction.csv')
    spiderManNoWayHome = pd.read_csv('SpiderMan No Way Home.csv')
    theAvengers = pd.read_csv('The Avengers.csv')
    theDarkKnight = pd.read_csv('The Dark Knight.csv')
    thorRagnarok = pd.read_csv('Thor Ragnarok.csv')
    df = pd.concat([avengersEndgame, forrestGump, johnWickChapterThree, joker, morbius, pulpFiction, spiderManNoWayHome, theAvengers, theDarkKnight, thorRagnarok], ignore_index=True)
    df = df.rename(columns = {"User's Rating out of 10":"Rating"})
    df.dropna(inplace = True)
    df['Review'] = df['Review'].astype(str)
    df['Review Title'] = df['Review Title'].astype(str)
    df['Rating'] = df['Rating'].str.strip()
    df = df[df.Rating != "Was this review helpful?  Sign in to vote."]
    df['Rating'] = df['Rating'].astype(int)
    return df

In [3]:
df = makeAndCleanDF ()

In [4]:
def getListOfDF ():
    listOfJSONS = ['part-01.json', 'part-02.json', 'part-03.json', 'part-04.json', 'part-05.json', 'part-06.json', 'sample.json']
    listOfDF = []
    for JSON in listOfJSONS:
        listOfDF.append(pd.read_json(JSON))
    return listOfDF

In [5]:
listOfDF = getListOfDF()

In [6]:
def addOtherDataToDataFrame (listOfDF, df):
    df2 = pd.concat(listOfDF, ignore_index=True)
    df2.dropna(inplace = True)
    df2.reset_index(drop=True, inplace=True)
    helpfulness = []
    for row in range(len(df2)):
        helpful = df2["helpful"][row]
        template = {"Usefulness Vote": helpful[0], "Total Votes":helpful[1]}
        helpfulness.append(template)
    helpfulness = pd.DataFrame.from_dict(helpfulness)
    df2 = pd.concat([df2, helpfulness], axis=1)
    df2[['review_summary', 'review_detail', 'Usefulness Vote', 'Total Votes']] = df2[['review_summary', 'review_detail', 'Usefulness Vote', 'Total Votes']].astype(str)
    df2[['Usefulness Vote', 'Total Votes']] = df2[['Usefulness Vote', 'Total Votes']].replace(",","", regex=True)
    df2[['Usefulness Vote', 'Total Votes']] = df2[['Usefulness Vote', 'Total Votes']].astype(float)
    df2 = df2.drop(axis = 1, columns = ['review_id', 'movie', 'spoiler_tag', 'helpful'])
    df2 = df2.rename(columns = {"reviewer":"User", "rating":"Rating", "review_summary":"Review Title", "review_date":"Date of Review", "review_detail":"Review"})
    df = pd.concat([df, df2], ignore_index=True)
    return df

In [7]:
df = addOtherDataToDataFrame (listOfDF, df)

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
def dropNonUsefulRows (df):
    df = df.drop(df.loc[df['Total Votes'] == 0].index)
    df.reset_index(drop=True, inplace=True)
    for index, row in df.iterrows():
        if row['Usefulness Vote']/row['Total Votes'] <= 0.5:
            df = df.drop(index)   
    return df

In [10]:
#df = dropNonUsefulRows (df)

In [11]:
def fixPunctuation (allTexts):
    allTexts = allTexts.replace(".", " . ")
    allTexts = allTexts.replace(",", " , ")
    allTexts = allTexts.replace("?", " ? ")
    allTexts = allTexts.replace("!", " ! ")
    allTexts = allTexts.replace(":", " : ")
    allTexts = allTexts.replace(";", " ; ")
    allTexts = allTexts.replace("/", " / ")
    allTexts = allTexts.replace(")", " ) ")
    allTexts = allTexts.replace("(", " ( ")
    allTexts = allTexts.replace("'", " ' ")
    allTexts = allTexts.replace("'", " ' ")
    
    
    return allTexts

In [12]:
def makeColumns (df):
    allTexts = ""
    reviewTitle = ' '.join(df["Review Title"])
    review = ' '.join(df["Review"])
    allTexts = reviewTitle + " " + review
    
    allTexts = allTexts.lower()
    
    allTexts = fixPunctuation (allTexts)
        
    allTexts = allTexts.split()
    
    dic = {}
        
    for word in allTexts:
        if word in dic:
            x += 1
        else:
            x = 1
    
        dic[word] = x
        
    nonRelevantWords = [key for key, val in dic.items() if val <= 1]
    relevantWords = {key:[] for key, val in dic.items() if val > 1}
    
    relevantWords = pd.DataFrame.from_dict(relevantWords)
    
    df = pd.concat([df, relevantWords], ignore_index=True)
    
    return df, nonRelevantWords

In [ ]:
df, nonRelevantWords = makeColumns (df)

In [ ]:
def getValues (df, nonRelevantWords):
    end = len(df)
    
    for i in range (end):
        wordsForRow = df["Review Title"][i] + " " + df["Review"][i]
        
        wordsForRow = wordsForRow.lower()
    
        wordsForRow = fixPunctuation (wordsForRow)
        
        wordsForRow = wordsForRow.split()
        
        unrepeatedWords = list(set(wordsForRow))
        
        for word in unrepeatedWords:
            if word not in nonRelevantWords:
                df.loc[i, word] = wordsForRow.count(word)
            else:
                pass
    
    df = df.fillna(0)
    
    return df

In [ ]:
df = getValues (df, nonRelevantWords)

In [ ]:
df

In [ ]:
cor = df.corr()

In [ ]:
cor

In [ ]:
for column in cor:
    if not(abs(cor['Rating'][column]) >= 0.01):
        df = df.drop(columns = column)

In [ ]:
df